# Statistiques et décompte des valeurs `Q0`
Décompte des articles annotés et des valeurs `Q0`.

`Q0` indique une localité que nous n'avons pas pu trouver

Auteur : Pierre Nugues

In [1]:
import json
import pandas as pd
from collections import Counter

In [2]:
with open('diderot_1751_wd_extraction.json', 'r') as f:
    diderot_wd = json.loads(f.read())

Article de géographie simple

In [3]:
diderot_wd[0]

{'vedette': 'A',
 'entreeid': 'v1-9-0',
 'texte': '*\u200b A, s. petite riviere de France, qui a sa source près de Fontaines en Sologne.',
 'qid': ['Q15803785'],
 'note': 'https://fr.wikisource.org/wiki/Page:Trevoux_-_Dictionnaire,_1743,_T01,_A.djvu/32',
 'wd_statements': [{'desc_en': ['river in France'],
   'desc_fr': ['rivière française'],
   'libellé_en': ['Conon'],
   'libellé_fr': ['Conon'],
   'type': ['Q4022'],
   'geo': ['Point(1.4019444 47.53333282)']}]}

Article de géographie avec des biographies

In [4]:
diderot_wd[4909]

{'vedette': 'GRENOBLE',
 'entreeid': 'v7-1475-0',
 'texte': 'GRENOBLE, Gratianopolis, (Géogr.)\u200b ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, & un parlement érigé en 1493 par Louis XI. qui n’étoit encore que dauphin ; mais son pere ratifia cette érection deux ans après.\n\n\nGrenoble est sur l’Isere, à onze lieues S O. de Chambéri, quarante-deux N. O. de Turin, seize S. E. de Vienne, cent vingt-quatre S. O. de Paris. Long. suivant Harris, 23d. 31′. 15″. suivant Cassini, 23d. 14′. 15″. latit 45d. 11′.\n\n\nCette ville reçut le nom de Gratianopolis de l’empereur Gratien fils de Valentinien I. car elle s’appelloit auparavant Cularo ; & c’est sous ce nom qu’il en est parlé dans une lettre de Plancus à Cicéron, epist. xxiij. Long-tems après, les Romains l’érigerent en cité : dans le cinquieme siecle, elle fut assujettie par les Bourguignons, & dans le sixieme par les François Mérovingiens ; ensuite elle a obéi à Lothaire, à Boson, à Charles le Gro

## Décompte des `Q0`

In [5]:
len(diderot_wd)

15274

In [6]:
cnt_articles = len(diderot_wd)
cnt_articles_avec_qid = 0
cnt_qid = 0
cnt_article_avec_qid0 = 0
cnt_qid0 = 0

for article in diderot_wd:
    if 'qid' in article:
        cnt_articles_avec_qid += 1
        cnt_qid += len(article['qid'])
        if 'Q0' in article['qid']:
            cnt_article_avec_qid0 += 1
            cnt_qid0 += article['qid'].count('Q0')

Pourcentages d'articles annotés

In [7]:
cnt_articles_avec_qid, cnt_articles, cnt_articles_avec_qid / cnt_articles

(10614, 15274, 0.6949063768495483)

Pourcentage des articles avec au moins un `Q0`

In [8]:
cnt_article_avec_qid0, cnt_articles_avec_qid, cnt_article_avec_qid0/cnt_articles_avec_qid

(222, 10614, 0.020915771622385528)

Pourcentage des `Q0` sur toutes les annotations

In [9]:
cnt_qid0, cnt_qid, cnt_qid0 / cnt_qid

(355, 13423, 0.02644714296356999)

Total des articles avec un `Q0`

In [10]:
cnt_article_avec_qid0/cnt_articles_avec_qid

0.020915771622385528

## Statistiques complètes

In [11]:
nbr_ann = 0
nbr_geo_seul = 0
nrb_liens = 0
nrb_liens_geo_seul = 0
nbr_art_q0 = 0
nbr_q0 = 0
nbr_region = 0
nbr_art_avec_humains = 0
nbr_humains = 0
nbr_liens_avec_humains = 0
nbr_renvoi = 0

for article in diderot_wd:
    if 'qid' in article:
        nbr_ann += 1
        nrb_liens += len(article['qid'])
        if 'Q0' in article['qid']:
            nbr_art_q0 += 1
            nbr_q0 += article['qid'].count('Q0')
        pas_dhumains = True
        if 'wd_statements' in article:
            for sous_article in article['wd_statements']:  # Nombre d'humains
                if 'type' in sous_article and 'Q5' in sous_article['type']:
                    nbr_humains += 1

            # Nombre d'articles avec humains
            for sous_article in article['wd_statements']:
                if 'type' in sous_article and 'Q5' in sous_article['type']:
                    nbr_art_avec_humains += 1
                    nbr_liens_avec_humains += len(article['qid'])
                    pas_dhumains = False
                    break
        if pas_dhumains:
            nbr_geo_seul += 1
            nrb_liens_geo_seul += len(article['qid'])
    elif 'renvoi' in article:
        nbr_renvoi += 1
    elif 'qid_region' in article:
        nbr_region += 1
    else:
        print('Erreur !', article)

In [12]:
df = pd.DataFrame(
    [{'Annotés': nbr_ann, 'Liens': nrb_liens, 'Q0': nbr_art_q0, 'Q0 totaux': nbr_q0, 'Reste': nbr_region},
     {'Annotés': nbr_renvoi},
     {'Annotés': nbr_ann + nbr_renvoi},
     {'Annotés': nbr_art_avec_humains, 'Humains': nbr_humains,
         'Liens': nbr_liens_avec_humains, 'Reste': 0},
     {'Annotés': nbr_ann + nbr_renvoi - nbr_art_avec_humains},
     {'Annotés': nbr_geo_seul, 'Liens': nrb_liens_geo_seul},
     ],
    index=['Articles de géographie', 'Renvois', 'Total annotés et renvois', 'Articles contenants des biographies', 'Articles sans biographies', 'Articles sans biographies et sans renvoi'])

In [13]:
df

,Annotés,Liens,Q0,Q0 totaux,Reste,Humains
Articles de géographie,10614,13423.0,222.0,355.0,4589.0,NaN
Renvois,71,NaN,NaN,NaN,NaN,NaN
Total annotés et renvois,10685,NaN,NaN,NaN,NaN,NaN
Articles contenants des biographies,842,2666.0,NaN,NaN,0.0,1718.0
Articles sans biographies,9843,NaN,NaN,NaN,NaN,NaN
Articles sans biographies et sans renvoi,9772,10757.0,NaN,NaN,NaN,NaN


## Partition en régions des articles restant à annoter

In [14]:
regions = []
for i, article in enumerate(diderot_wd):
    if 'qid' in article:
        continue
    if 'renvoi' in article:
        continue
    if 'qid_region' in article:
        regions += article['qid_region']

In [15]:
cnt = Counter(regions)

In [16]:
len(cnt), cnt.most_common()

(32,
 [('Q41', 537),
  ('Q15', 493),
  ('Q51614', 476),
  ('Q38', 347),
  ('Q668', 274),
  ('Q794', 225),
  ('Q18', 192),
  ('Q49', 190),
  ('Q183', 158),
  ('Q28', 156),
  ('Q23522', 155),
  ('Q29', 154),
  ('Q23792', 125),
  ('Q36', 121),
  ('Q159', 115),
  ('Q142', 102),
  ('Q31945', 98),
  ('Q252', 89),
  ('Q148', 81),
  ('Q55', 74),
  ('Q115', 71),
  ('Q17', 65),
  ('Q11768', 61),
  ('Q21', 51),
  ('Q66747521', 50),
  ('Q309272', 44),
  ('Q22', 32),
  ('Q45', 23),
  ('Q39', 11),
  ('Q20', 8),
  ('Q408', 7),
  ('Q165', 4)])